In [ ]:
using ModelingToolkit
using MethodOfLines
using DifferentialEquations
using DomainSets
using ProgressMeter
using Plots

In [ ]:
struct FDMesh2D
    xl::Float64
    xr::Float64
    yl::Float64
    yr::Float64
    nx::Int64
    ny::Int64
    nodes::Vector{Vector{Float64}}
    nodes_idxs::CartesianIndices{2}
    #edge_midpoints::Vector{Vector{Float64}}
end

In [ ]:
function FDMesh2D(xl::Float64, xr::Float64, nx::Int64, yl::Float64, yr::Float64, ny::Int64)
    hx = (xr - xl) / nx
    hy = (yr - yl) / ny
    metrics = [hx, hy]
    nodes_idxs = CartesianIndices((nx+1, ny+1))
    nodes = [(collect(Tuple(idx)).-1) .* metrics for idx in nodes_idxs] |> vec
    println(typeof(nodes))
    return FDMesh2D(xl, xr, yl, yr, nx, ny, nodes, nodes_idxs)
end  

In [ ]:
xl, xr = 0.0, 1.0
yl, yr = 0.0, 1.0
nx, ny = 10, 10
mesh = FDMesh2D(xl ,xr, nx, yl, yr, ny)
println(mesh.nodes[2])
println(mesh.nodes_idxs[2])